In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import accuracy_score
import numpy as np
import optuna

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class RNNClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(RNNClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.rnn(x, h0)
        out = out[:, -1, :]
        out = self.fc(out)
        return out

def load_data(file_name):
    with np.load(file_name) as data:
        data_array = data['data']
        labels_array = data['labels']
    return data_array, labels_array

def to_tensor(data, labels):
    data_tensor = torch.Tensor(data)
    labels_tensor = torch.LongTensor(labels.argmax(axis=1))
    return data_tensor, labels_tensor

train_data, train_labels = load_data('trainset_normalized.npz')
test_data, test_labels = load_data('testset_normalized.npz')

train_data_tensor, train_labels_tensor = to_tensor(train_data, train_labels)
test_data_tensor, test_labels_tensor = to_tensor(test_data, test_labels)

input_size = 16  
output_size = 5  

def train_and_evaluate(model, train_loader, test_loader, optimizer, criterion, num_epochs):
    for epoch in range(num_epochs):
        model.train()
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    model.eval()
    with torch.no_grad():
        all_labels = []
        all_preds = []
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(predicted.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    return accuracy

def objective(trial):

    batch_size = int(trial.suggest_categorical('batch_size', [16, 32, 64, 128]))
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-6, 1e-2)
    hidden_size = int(trial.suggest_categorical('hidden_size', [32, 64, 128, 256, 512, 1024]))
    num_layers = trial.suggest_int('num_layers', 1, 7)
    num_epochs = int(trial.suggest_categorical('num_epochs', [10, 15,20, 25,30,40]))  

    train_loader = DataLoader(TensorDataset(train_data_tensor, train_labels_tensor), batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(TensorDataset(test_data_tensor, test_labels_tensor), batch_size=batch_size, shuffle=False)

    model = RNNClassifier(input_size, hidden_size, output_size, num_layers).to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    accuracy = train_and_evaluate(model, train_loader, test_loader, optimizer, criterion, num_epochs)
    return accuracy

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=1000)

print("最佳参数: ", study.best_params)


[I 2024-01-01 18:41:51,152] A new study created in memory with name: no-name-efed4c94-f7f2-4c2d-b6b1-24da06ce892f
C:\Users\wenjie\AppData\Local\Temp\ipykernel_11360\1339970873.py:74: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-6, 1e-2)
[I 2024-01-01 18:43:21,535] Trial 0 finished with value: 0.3224489795918367 and parameters: {'batch_size': 16, 'learning_rate': 0.0014741805111190688, 'hidden_size': 32, 'num_layers': 4, 'num_epochs': 20}. Best is trial 0 with value: 0.3224489795918367.
C:\Users\wenjie\AppData\Local\Temp\ipykernel_11360\1339970873.py:74: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_r

KeyboardInterrupt: 